In [ ]:
import math
import numpy as np

def crossvalidation5x2(df,seeds,column,num_dim=4):
    
    f = lambda x: df[df['embedding'].str.match(x)][[column]].values
    p1 = np.zeros((len(seeds),num_dim))
    p2 = np.zeros((len(seeds),num_dim))

    for i,seed in enumerate(seeds):
        #order by seed
        #for each seed, there are 4 (one for each dimension), "a" and 4 "b" for a given model
        #5 seeds total, thus 20 entries per model
        custom_a_str ="25full.+"+seed+"a"
        custom_b_str = "25full.+"+seed+"b"
        glove_a_str="glove.+"+seed+"a"
        glove_b_str="glove.+"+seed+"b"
        
        if i == 0:
            print(f(custom_a_str))
            print(f(glove_a_str))
        
        
        p1[i] = np.squeeze(f(custom_a_str)-f(glove_a_str))
        p2[i] = np.squeeze(f(custom_b_str)-f(glove_b_str))

    pmean = (p1+p2)/2
    variance_list = (p1-pmean)**2+(p2-pmean)**2
    variance_means = variance_list.mean(axis=0)
    t_list = np.divide(p1[0],np.sqrt(variance_means))
    return p1,pmean,variance_means,t_list

In [ ]:
#before running below, load results dataframe with accuracy and ham from model into a dataframe df

In [ ]:
seeds = ['6742404301','491643334','2583619689','9051856379','6961591582']

#use t_list in R to determine p-value, will need to run twice: with column = "accuracy" and "ham"
#for two tailed t-test with 5 dof -> pt(c(t_list),5,lower=FALSE)*2 to get p-value for accuracy t_list, lower=TRUE for Hamming
p1,pmean,variance_means, t_list = crossvalidation5x2(df,seeds,'accuracy')

# Error analysis

In [ ]:
import pickle
import pandas as pd

class ErrorAnalysis:
    def __init__(self,df,seed,dimensions,models):
        self.modelstr_list=list()
        self.seed = seed
        self.df_seed_dict = dict()
        self.df_seed_dict_inc = dict()
        
        for model in models:
            for dim in dimensions:
                modelstr=model+'.'+str(dim)+'d_'+seed
                self.modelstr_list.append(modelstr)
                df_match = df[df['embedding'].str.match(modelstr)]
                self.df_seed_dict[modelstr]= df_match
                self.df_seed_dict_inc[modelstr]=self.filter_incorrect(df_match)
        
    def filter_incorrect(self,df):
        
        df_incorrect = pd.DataFrame(columns=df.columns)
        
        for i,row in df.iterrows():
#             #correction for v2 where misc is assigned to zero vector but has no model output
#             if not row['pred']:                
#                 row['pred']='Misc'
            
            if set(row['truth']) == set(row['pred']):
                continue
            else:
                df_incorrect = df_incorrect.append(row)
                
        return df_incorrect
                
    def mass_csv_export(self,savestr):
        writer_full = pd.ExcelWriter(savestr+'_full.xlsx',engine='xlsxwriter')
        writer_inc =  pd.ExcelWriter(savestr+'_incorrect.xlsx',engine='xlsxwriter')
        
        for modelstr in self.modelstr_list:
            self.df_seed_dict[modelstr].to_excel(writer_full,sheet_name=modelstr)
            self.df_seed_dict_inc[modelstr].to_excel(writer_inc,sheet_name=modelstr)
        
        writer_full.save()
        writer_inc.save()
    
    def create_label_dict(self,df):
        label_numbers = {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 19}
        self.df_bylabel = dict()
        for label_num in label_numbers :
            self.df_bylabel[label_num] = []
            
        for index,row in df.iterrows():
            self.df_bylabel[row['truth'].count(",")+1].append(row)
    
    def correct_count(self):
        label_correct_count=dict()
        label_total_count=dict()

        for key in self.df_bylabel.keys():
            label_correct_count[key]=0
            label_total_count[key]=0
            
            for item in self.df_bylabel[key]:
                if(item['truth']==item['pred']):
                    label_correct_count[key]+=1
                label_total_count[key]+=1
        return label_correct_count,label_total_count

In [ ]:
#df_filepath will be path to error analysis pkl from model output

with open(df_filepath,"rb") as file:
             analysis = ErrorAnalysis(pickle.load(file),
                         '6742404301a',[50,100,200,300],['glove','25full'])

In [ ]:
for modelstr in analysis.modelstr_list:
    empty_pred = 0
    for pred in analysis.df_seed_dict[modelstr]['pred']:
        if not pred:
            empty_pred+=1
    print(modelstr)
    print(empty_pred)